In [6]:
import numpy as np
import pandas as pd

In [7]:
df=pd.read_csv(r'C:\Users\My PC\Desktop\Machine Learning\NLP\spam.csv',encoding='latin-1')

In [8]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [9]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

In [10]:
blanks=[]
for i,lb,sp in df.itertuples():
    if type(sp)==str:
        if sp.isspace():
            blanks.append(i)

In [11]:
df.drop(blanks,inplace=True)

In [12]:
df.isnull().sum()

v1    0
v2    0
dtype: int64

In [21]:
from sklearn.svm import LinearSVC
clf=LinearSVC()

In [13]:
df['v1'].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [14]:
x=df['v2']
y=df['v1']

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
cv=CountVectorizer()

In [19]:
cv.fit(x_train)
xtc=cv.transform(x_train)

In [20]:
xtc


<3900x7206 sparse matrix of type '<class 'numpy.int64'>'
	with 51569 stored elements in Compressed Sparse Row format>

In [26]:
from sklearn.feature_extraction.text import TfidfTransformer

In [27]:
tf=TfidfTransformer()

In [28]:
xtf=tf.fit_transform(xtc)

In [29]:
xtf

<3900x7206 sparse matrix of type '<class 'numpy.float64'>'
	with 51569 stored elements in Compressed Sparse Row format>

In [30]:
xtf.shape

(3900, 7206)

We can use TfidfVectorizer to perform fit and transform at same time 

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
tfv=TfidfVectorizer()

In [33]:
xtf2=tfv.fit_transform(x_train)

In [34]:
xtf2.shape

(3900, 7206)

In [36]:
from sklearn.pipeline import Pipeline

In [37]:
vex=Pipeline([('Vect',TfidfVectorizer()),('Model',LinearSVC())])

In [38]:
vex.fit(x_train,y_train)

Pipeline(steps=[('Vect', TfidfVectorizer()), ('Model', LinearSVC())])

In [39]:
pred=vex.predict(x_test)

In [40]:
from sklearn.metrics import confusion_matrix, classification_report

In [41]:
print(confusion_matrix(pred,y_test))

[[1450   25]
 [   3  194]]


In [42]:
print(classification_report(pred,y_test))

              precision    recall  f1-score   support

         ham       1.00      0.98      0.99      1475
        spam       0.89      0.98      0.93       197

    accuracy                           0.98      1672
   macro avg       0.94      0.98      0.96      1672
weighted avg       0.98      0.98      0.98      1672



<h1>Using nltk</h1>

In [45]:
data=pd.read_csv(r'C:\Users\My PC\Desktop\Machine Learning\NLP\spam.csv',encoding='latin-1')

In [48]:
data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

In [49]:
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [52]:
import nltk 
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [53]:
ps=PorterStemmer()

In [56]:
sent=[]
for i in range(0,len(data)):
    r=re.sub('[^a-zA-Z]',' ',data['v2'][i])
    r=r.lower()
    r=r.split()
    r=[ps.stem(word) for word in r if not word in stopwords.words('english')]
    r=' '.join(r)
    sent.append(r)
               
    

In [57]:
sent[:5]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though']

In [58]:

X=cv.fit_transform(sent).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [71]:
Y=pd.get_dummies(data['v1'])
Y=Y.iloc[:,1].values

In [72]:
Y

array([0, 0, 1, ..., 0, 0, 0], dtype=uint8)

In [73]:
from sklearn.model_selection import train_test_split

In [74]:
xtrainn,xtestn,ytrainn,ytestn=train_test_split(X,Y,test_size=0.3,random_state=112)

In [75]:
from sklearn.naive_bayes import MultinomialNB

In [76]:
model=MultinomialNB().fit(xtrainn,ytrainn)

In [77]:
ypred=model.predict(xtestn)